In [42]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
#from sklearn.

data=pd.read_csv('../input/heartdisease-2/heart_disease.csv')
labels=data.values[:,-1]
labels[labels>1]=1
labels=labels.astype(int)

data=data.values[:,:-1]

In [43]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier

In [44]:
num_estimator = 5
cv = KFold(n_splits=5)
base_learners = [
                 ('dt_1', DecisionTreeClassifier(random_state=0, criterion='gini', splitter='random')),
                 ('dt_2', DecisionTreeClassifier(random_state=0, criterion='gini', splitter='best')),
                 ('dt_3', DecisionTreeClassifier(random_state=0, criterion='entropy', splitter='random')),
                 ('dt_4', DecisionTreeClassifier(random_state=0, criterion='entropy', splitter='best')),
                 ('dt_5', DecisionTreeClassifier(random_state=42, criterion='entropy', splitter='best'))
                ]
final_estimator = DecisionTreeClassifier(random_state=0, criterion='entropy', splitter='best')

In [45]:
%%time
stack_model = StackingClassifier(estimators=base_learners, final_estimator=final_estimator)
stack_scores = cross_validate(stack_model, data, labels, scoring='accuracy', cv=cv, return_train_score=True)
print('%i fold Stacking Accuracy train score: %.3f (%.3f)' % (5, np.mean(stack_scores['train_score']), np.std(stack_scores['train_score'])))
print('%i fold Stacking Accuracy test score: %.3f (%.3f)' % (5, np.mean(stack_scores['test_score']), np.std(stack_scores['test_score'])))

5 fold Stacking Accuracy train score: 1.000 (0.000)
5 fold Stacking Accuracy test score: 0.769 (0.032)
CPU times: user 354 ms, sys: 2.63 ms, total: 356 ms
Wall time: 354 ms


In [46]:
%%time
rf_model = RandomForestClassifier(n_estimators=num_estimator)
rf_scores = cross_validate(rf_model, data, labels, scoring='accuracy', cv=cv, return_train_score=True)
print('%i fold RandomForest Accuracy train score: %.3f (%.3f)' % (5, np.mean(rf_scores['train_score']), np.std(rf_scores['train_score'])))
print('%i fold RandomForest Accuracy test score: %.3f (%.3f)' % (5, np.mean(rf_scores['test_score']), np.std(rf_scores['test_score'])))

5 fold RandomForest Accuracy train score: 0.975 (0.010)
5 fold RandomForest Accuracy test score: 0.812 (0.031)
CPU times: user 69.3 ms, sys: 2.88 ms, total: 72.1 ms
Wall time: 71.5 ms


In [47]:
%%time
ab_model = AdaBoostClassifier(n_estimators=num_estimator)
ab_scores = cross_validate(ab_model, data, labels, scoring='accuracy', cv=cv, return_train_score=True)
print('%i fold AdaBoost Accuracy train score: %.3f (%.3f)' % (5, np.mean(ab_scores['train_score']), np.std(ab_scores['train_score'])))
print('%i fold AdaBoost Accuracy test score: %.3f (%.3f)' % (5, np.mean(ab_scores['test_score']), np.std(ab_scores['test_score'])))

5 fold AdaBoost Accuracy train score: 0.845 (0.008)
5 fold AdaBoost Accuracy test score: 0.841 (0.049)
CPU times: user 64.5 ms, sys: 0 ns, total: 64.5 ms
Wall time: 64.8 ms


* Mean of accuracy is the best for AdaBoost with 5 estimator regarding the test accuracy, standard deviation for RF but there the mean is lower
* For memory: AdaBoost using less memory, than the rest
* For speed: RandomForest and Stacking can be parallelized, but AdaBoost was faster for me
* For performance: AdaBoost (with 50 estimator for AB and RF, RF was better)